<a href="https://colab.research.google.com/github/KNGLJordan/CDMO-project/blob/main/src/SAT/sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt install python3-z3

[sudo] password for lucazini03: 


In [3]:
from z3 import *
from itertools import combinations

In [4]:
#the man the myth the legend: angelo quarta

#NAIVE PAIRWISE
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

def at_least_k_np(bool_vars, k):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k)

def at_most_k_np(bool_vars, k):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k):
    return And(at_most_k_np(bool_vars, k), at_least_k_np(bool_vars, k))


#SEQUENTIAL
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

#HEULE
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

#BINARY
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{name}_{i}") for i in range(m)]
    binaries = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binaries[i][j] == "1":
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))        
    return And(constraints)

def exactly_one_bw(bool_vars, name):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name)) 

In [7]:
from z3 import *
import time
from itertools import combinations
import numpy as np

def sports_scheduling_sat(n, timeout=300):
    start_time = time.time()
    weeks = n - 1
    periods = n // 2
    teams = list(range(n))

    home = np.empty((n, weeks, periods), dtype=object)
    away = np.empty((n, weeks, periods), dtype=object)
    for t in teams:
        for w in range(weeks):
            for p in range(periods):
                home[t, w, p] = Bool(f"H_{t}_{w}_{p}")
                away[t, w, p] = Bool(f"A_{t}_{w}_{p}")

    def build_solver(imbalance):
        s = Solver()
        s.set("timeout", int(timeout * 1000 - (time.time() - start_time) * 1000))

        # 0. Each slot has exactly one home and one away
        for w in range(weeks):
            for p in range(periods):
                s.add(exactly_one_seq([home[t, w, p] for t in teams], name=f"slot_home_{w}_{p}"))
                s.add(exactly_one_seq([away[t, w, p] for t in teams], name=f"slot_away_{w}_{p}"))

        # 1. Every team plays exactly once per week
        for t in teams:
            for w in range(weeks):
                vars_in_week = [home[t, w, p] for p in range(periods)] + \
                               [away[t, w, p] for p in range(periods)]
                s.add(exactly_one_seq(vars_in_week, name=f"team_plays_{t}_{w}"))

        # 2. Every pair meets exactly once
        for i, j in combinations(teams, 2):
            match_slots = []
            for w in range(weeks):
                for p in range(periods):
                    match_slots.append(
                        Or(And(home[i, w, p], away[j, w, p]),
                           And(home[j, w, p], away[i, w, p]))
                    )
            s.add(exactly_one_seq(match_slots, name=f"pair_meets_{i}_{j}"))

        # 3. At most 2 games per team per period
        for t in teams:
            for p in range(periods):
                games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
                s.add(at_most_k_seq(games, 2, name=f"team_{t}_period_{p}"))

        # 4. Home/away balance: each team has home games in [min_home, max_home]
        min_home = (weeks // 2) - imbalance
        max_home = (weeks + 1) // 2 + imbalance
        for t in teams:
            home_games = [home[t, w, p] for w in range(weeks) for p in range(periods)]
            s.add(at_least_k_seq(home_games, min_home, f"home_min_{t}"))
            s.add(at_most_k_seq(home_games, max_home, f"home_max_{t}"))
        return s

    # Binary search for minimum imbalance
    # 1. Set the initial search interval for imbalance
    low, high = 0, weeks // 2
    """ Example with n = 6, weeks = 5
            Imbalance 0 (perfect balance): Each team has 2 or 3 home games.
            Imbalance 1: Each team has 1, 2, 3, or 4 home games.
            Imbalance 2: Each team has 0, 1, 2, 3, 4, or 5 home games.
                e.g. above min_home will be (5//2)-2=0
                           max_home will be (5+1)//2+2=5  """

    best_model = None

    # 2. Repeat while the interval is valid and time remains
    while low <= high and (time.time() - start_time) < timeout:
        mid = (low + high) // 2  # 3. Pick the midpoint (current imbalance to try)
        s = build_solver(mid)    # 4. Build the model with this imbalance
        if s.check() == sat:     # 5. If a solution exists...
            best_model = s.model()
            high = mid - 1       # 6. ...try a smaller imbalance (left half)
        else:
            low = mid + 1        # 7. ...otherwise, try a larger imbalance (right half)

    elapsed = time.time() - start_time
    if best_model is None:
        print("No solution found.")
        return None, elapsed, None

    # Extract schedule from best_model
    sched = [[None]*weeks for _ in range(periods)]
    for w in range(weeks):
        for p in range(periods):
            for i in teams:
                if is_true(best_model.evaluate(home[i, w, p])):
                    for j in teams:
                        if is_true(best_model.evaluate(away[j, w, p])):
                            sched[p][w] = (i+1, j+1)
    return sched, elapsed, best_model

# Prompt for input
n = int(input("Enter even number of teams: "))

sched, elapsed, m = sports_scheduling_sat(n, timeout=300)

if sched:
    weeks = len(sched[0])
    print("        ", end="")
    for w in range(1, weeks+1):
        print(f"Week {w}".center(10), end="")
    print()
    for p, row in enumerate(sched, 1):
        print(f"Period {p}:", end=" ")
        for game in row:
            print(f"{game[0]} v {game[1]}".center(10), end="")
        print()
    print(f"\nExecution time: {elapsed:.2f} seconds")

    # Print home/away counts for each team
    total_imbalance = 0
    for t in range(n):
        home_count = sum(1 for w in range(weeks) for p in range(len(sched)) if sched[p][w] and sched[p][w][0] == t + 1)
        away_count = sum(1 for w in range(weeks) for p in range(len(sched)) if sched[p][w] and sched[p][w][1] == t + 1)
        print(f"Team {t + 1}: Home {home_count}, Away {away_count}")
        total_imbalance += abs(home_count - away_count)
    print(f"Total imbalance: {total_imbalance - n} (should be 0)")

          Week 1    Week 2    Week 3    Week 4    Week 5    Week 6    Week 7    Week 8    Week 9   Week 10   Week 11  
Period 1:   12 v 3    1 v 8     7 v 11    9 v 4     11 v 1    4 v 10    10 v 6    2 v 6     2 v 3     5 v 12    9 v 5   
Period 2:   4 v 8     5 v 2     1 v 12    6 v 5     12 v 8    1 v 9     11 v 4    10 v 7   11 v 10    9 v 3     7 v 6   
Period 3:   6 v 11    6 v 3     5 v 3     8 v 2     10 v 2    11 v 5    12 v 7    8 v 9     12 v 9    7 v 1     4 v 1   
Period 4:   1 v 2    12 v 10    9 v 10    3 v 7     7 v 5     3 v 8     2 v 9     4 v 12    1 v 6     6 v 4     8 v 11  
Period 5:   7 v 9     9 v 11    4 v 2     10 v 1    3 v 4     12 v 6    5 v 8     1 v 5     8 v 7     2 v 11    10 v 3  
Period 6:   5 v 10    4 v 7     6 v 8    11 v 12    6 v 9     7 v 2     3 v 1     3 v 11    5 v 4     8 v 10    2 v 12  

Execution time: 43.84 seconds
Team 1: Home 6, Away 5
Team 2: Home 5, Away 6
Team 3: Home 5, Away 6
Team 4: Home 6, Away 5
Team 5: Home 6, Away 5
Team 6: H